In [186]:
import numpy as np
import sympy as sym
import pandas as pd

In [498]:
s,t = sym.symbols('s t', real = True)
x1,x2,x3 = sym.symbols('x1 x2 x3')
A=np.array([[3, 10],[5,2]])
B=np.array([1,1])
C=np.array([1,0])
D=np.array([0])




In [444]:
def numpy2sympy(M):
    return sym.Matrix(M)

def ss2tf(A,B,C,D):
    '''this method converts from state space to transfer function
    inputs: A Matrix, B Matrix, C Matrix, D Matrix
    returns: transfer function'''
    A,B,C,D=numpy2sympy(A),numpy2sympy(B),numpy2sympy(C),numpy2sympy(D)   
    TF=C.transpose() * (sym.eye(A.shape[0])*s-A).inv() * B + sym.Matrix([D])
    TF=sym.simplify(TF)
    return TF
ss2tf(A,B,C,D)


Matrix([[(s + 8)/(s**2 - 5*s - 44)]])

In [446]:
A=np.array([[0, 1,0],[0,0,1],[-6,-11,-6]])
def isStable(A):
    '''checks whether a system is stable or not
    Parameters: A Matrix
    return True or False'''
    A=numpy2sympy(A)
    for i in A.eigenvals().keys():
        if i>0:
            return False
        else:
            return True
isStable(A)


True

In [448]:
A=np.array([[0, 1],[-1,-2]])
C=np.array([1,0])

def isObservable(A,C):
    '''checks whether a system is observable or not
    Parameters: A Matrix, C Matrix
    return True or False'''
    A,C=numpy2sympy(A),numpy2sympy(C)
    n=A.shape[0]        
    O=C.transpose()
    for i in range(n-1):
        O=O.row_insert(i+1,C.transpose()*A**(i+1))
    if O.rank()==n:
        return True
    else :
        return False

isObservable(A,C)



True

In [449]:
A=np.array([[0, 1],[-1,-2]])
B=np.array([0,1])

def isControllable(A,B):
    '''checks whether a system is controllable or not
    Parameters: A Matrix, B Matrix
    return True or False'''
    A,B=numpy2sympy(A),numpy2sympy(B) 
    n=A.shape[0]
    c=B
    for i in range(n-1):
        c=c.col_insert(i+1,A**(i+1)*B)
        
    if c.rank()==n:
        return True
    else :
        return False

    
isControllable(A,B)

True

In [450]:
def tf2ss(num,den):
    pass

In [511]:
#take too much time to inverse laplace
A=np.array([[3, 10],[5,2]])
B=np.array([1,1])
x0=np.array([1,0])
u=1

def solve(A,B,x0,u):
    '''finds the system solution in S domain
    Parameters: A Matrix ,B Matrix,initial condtion Matrix,input u
    returns: system solution in time domain
    '''
    A,B,x0=numpy2sympy(A),numpy2sympy(B),numpy2sympy(x0)
    u=sym.laplace_transform(u, t, s, noconds=True)
    x=(sym.eye(A.shape[0])*s-A).inv()*x0 +(sym.eye(A.shape[0])*s-A).inv()*B*u
    x=sym.simplify(x)

    return x


In [512]:
solve(A,B,x0,u)

Matrix([
[(s**2 - s + 8)/(s*(s**2 - 5*s - 44))],
[   2*(3*s + 1)/(s*(s**2 - 5*s - 44))]])

In [513]:
def roots2poly(*roots):
    from sympy import Symbol
    x = Symbol('x')
    whole =1
    for root in roots:
        whole *=(x-root)
    p=sym.poly(whole.expand(),x)
    p=p.all_coeffs()
    return p 


def placePoles(A,B,p):
    '''
    finds gain k for pole placplacement using Ackermann's method
    Parameters: A Matrix,B Matrix, poles
    returns: gain vector
    
    '''
    A,B=numpy2sympy(A),numpy2sympy(B)  

    arr=np.zeros((1,A.shape[0]-1))
    arr=sym.Matrix(np.append(arr,[1])).transpose()
    
    
    n=A.shape[0]
    c=B
    for i in range(n-1):
        c=c.col_insert(i+1,A**(i+1)*B)
    
    Q=sym.zeros(A.shape[0])
    for i in range(len(p)):
        Q=Q+p[i]*A**(len(p)-1-i)
    
    return (arr * c.inv() * Q)  

A=np.array([[0, 1,0],[0,0,1],[-2,-3,-5]])
B=np.array([0,0,1])
p=roots2poly(-4+3j,-4-3j,-4)





In [514]:
k=placePoles(A,B,p)
k

Matrix([[98.0, 54.0, 7.0]])

In [463]:

n0=np.array([2,1,1,4])
d0=np.array([5,1,3,6])
n=n0/d0[0]
d=d0/d0[0]
A=np.zeros((len(d)-1,len(d)-1))
A[:,0]=-1*d[1:]
A[0:len(d)-2,1:]=np.eye(len(d)-2)


C=np.zeros(len(d)-1)
C[0]=1/d0[0]

D=np.array([n[0]])

In [464]:
ss2tf(A,B,C,D)

Matrix([[(0.144*s**5 + 0.0575999999999999*s**4 + 0.17856*s**3 + 0.27936*s**2 + 0.1008*s + 0.14688)/(0.36*s**5 + 0.144*s**4 + 0.4464*s**3 + 0.5184*s**2 + 0.216*s + 0.2592)]])

(array([[-0.2,  1. ,  0. ],
        [-0.6,  0. ,  1. ],
        [-1.2,  0. ,  0. ]]),
 array([0.2, 0. , 0. ]),
 array([0.4]))